Sistemas de Recomendación 
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/statistics-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/statistics-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 

---

# Definición del problema real

Un problema para los retailers es poder recomendar productos afines a sus compradores basados en el histórico general de las ventas tal que se puedan sugerir en la compra actual productos que podría haber olvidado. Nótese que acá no se tienen en cuenta las compras realizadas antes por un determinado cliente, si no los productos que usualmente se llevan juntos.   

# Definición del problema en términos de los datos

Se tiene una muestra de las transacciones de venta realizadas donde cada fila representa una trasacción y las variables $x_i$ representan los productos:


     #  Productos
    --------------------
     1  x1, x2, x3
     2  x1, x3
     3  x2, x4
     4  x1, x2, x3
     5  x1, x2, x5
     6  x1, x2, x3, x4
     7  x4, x6
     8  x1, x2, x4, x6
     9  x1, x3, x4
    10  x1, x2, x3
   

Si un cliente compra $x_1$ y $x_3$ que producto o productos se le deben recomendar?

# Solución

## Planteamiento del problema

Una transacción se representa a través del conjunto de ítems comprados en ella. Así, una transacción con cuatro ítems se representa como:

$$\{x_1, x_2, x_3, x_4 \}$$

El problema descrito equivale a determinar los productos $x_i$, con $x_i \notin \{x_1, x_2, x_3, x_4 \}$, con mayor probabilidad de ser comprados sabiendo que el cliente ya seleccionó $x_1$, $x_2$, $x_3$ y $x_4$. Dicho de otra forma, se deben computar las probabilidades condicionales $\text{Pr}(x_i \, | \,  x_1, x_2, x_3, x_4)$ y recomendar los $N$ productos $x_i$ con mayor probabilidad condicional de compra. En este problema se asume que SI existe una dependencia entre la compra de un producto y otro; si esta dependencia no existe, no tiene sentido construir el sistema de recomendación.

De esta forma, el objetivo del sistema de recomendación es construir una regla de asociación de la forma:

$$\{x_2, x_3, x_4 \} \rightarrow x_1$$

la cual indica que cuando se compran $x_2$, $x_3$ y $x_4$ también se compra (implica) $x_1$.

Es posible evaluar todas las posibles reglas por fuerza bruta (enumeración). Para el caso anterior, las reglas podrían ser: $\{x_2\} \rightarrow x_1$, $\{x_3\} \rightarrow x_1$, $\{x_4\} \rightarrow x_1$,  $\{x_1\} \rightarrow x_2$, $\{x_3\} \rightarrow x_2$, ..., 
$\{x_2, x_3\} \rightarrow x_1$, $...$, $\{x_1, x_2\} \rightarrow x_4$ y así sucesivamente, hasta construir todas las permutaciones posibles. Sin embargo, esta solución resulta imposible en términos prácticos debido a que la cantidad de reglas crece exponencialmente.

---

**Ejercicio.--** Cuántas reglas posibles hay para el caso anterior?

---

El algoritmo **Apriori** se basa en evaluar solamente las reglas que tienen una frecuencia alta (mayor probabilidad). El algoritmo se basa en la siguiente heurística: para que el conjunto $\{x_1, x_2\}$ sea frecuente (que tenga una probabilidad alta), los ítems $x_1$ y $x_2$ deben ser frecuentes; es decir, si $x_1$ o $x_2$ son infrecuentes, su combinación no es evaluada.  

Para medir la importancia de una regla se usa el soporte y la confianza. Si una regla de asocación se escribe como 

$$X \rightarrow Y$$

el soporte es la proporción de veces que $X$ aparece respecto al total de transacciones; nótese que acá se está hablando explícitamente de probabilidad. 

La confianza se define como:

$$\text{confidence}(X \rightarrow Y) = \frac{\text{support}(X, Y)}{\text{support}(X)} $$

la cual se interpreta como la cantidad de veces en que la presencia de $X$ resulta en la presencia de $Y$. Es decir, ya que se dio $X$ que tan probable es que se de $Y$.

---

**Ejercicio.--** Es verdad que $\text{confidence}(X \rightarrow Y) = \text{confidence}(Y \rightarrow X)$?

---

El algoritmo Apriori opera en dos fases: En la primera fase se identifican todos los conjuntos de ítems que cumplen con el soporte mínimo requerido (o probabilidad mínima de compra). En la segunda fase, con los ítems identificados en la fase uno, se crean reglas que cumplen con la confianza mínima requerida.

En la primera fase, se procede de forma constructiva de la siguiente forma (explícitamente se está calculado probabilidad):

* Se calcula el soporte de cada ítem y se seleccionan aquellos ítems que cumplen con el soporte mínimo requerido.


* Se forman todos las conjuntos (combinaciones) de dos ítems. Sólo se consideran combinaciones de dos ítems que contengan ítems que cumplen el soporte mínimo requerido. Se seleccionan aquellas combinaciones de dos ítems que cumplen con el soporte mínimo requerido.


* Se forman todos los conjuntos de tres ítems. No se consideran combinaciones que contengan conjuntos no frecuentes de dos ítems. Es decir, si la combinación {x1, x4} no se frecuente (no cumple con el soporte mínimo requerido), entonces no se consideran combinaciones como {x1, x3, x4} o {x1, x2, x4}.


* Se continua con las combinaciones de cuatro ítems y se seleccionan aquellas que cumplan con el soporte mínimo requerido.

---
**Ejercicio.--** Calcule el soporte para $x_1$, ..., $x_6$, esto es, la probabilidad de que $x_i$ aparezca en una transacción. Si el soporte mínimo requerido es del 40%, cuáles ítems debe seleccionarse?

         X   support   seleccionado
     --------------------------------
        x1         ?              ?
        x2      7/10             Si
        x3         ?              ?
        x4         ?              ?
        x5         ?              ?
        x6      2/10             No

---

**Ejercicio.--** Cuáles las combinaciones de dos ítems que deben ser consideradas de acuerdo con el resultado del ejercicio anterior? Cúales cumplen con un soporte mínimo del 40%?

              X   support   seleccionado
     -------------------------------------
       {x1, x2}      6/10             Si
              ?         ?              ?
              ?         ?              ?
              ?         ?              ?
       {x2, x4}      2/10             No
              ?         ?              ?
               
---

**Ejercicio.--** Cúales combinaciones de tres ítems pueden considerarse y cuáles cumplen con el soporte mínimo requerido?

---

En la segunda fase, para todas las combinaciones de 2, 3, ... ítems, se generan todas las reglas posibles y se evalúa su confianza. Se seleccionan aquellas que cumplen con la confianza mínima requerida. Si la combinación {x1, x2} cumplio con el soporte mínimo, entonces se consideran las reglas {x1} $\to$ {x2} y {x2} $\to$ {x1}. Nótese que es posible condierar reglas con varios elementos en el consecuente como por ejemplo {x1} $\to$ {x2, x3}.

El *lift* de una regla mide la importancia de una regla en relación a la tasa típica de compra, dado que se sabe que un ítem o un conjunto de ítems han sido comprados.

$$\text{lift}(X \rightarrow Y) = \frac{\text{confidence}(X, Y)}{\text{support}(Y)} $$


---
**Ejercicio.--** Para todas las combinaciones que cumplen con el soporte mínimo requerido, forme las correspodientes reglas y cómpute la confianza y el lift. Si la confianza mínima es del 70%, cuáles reglas deben ser seleccionadas?


     Combinación    Reglas           Confianza   Lift  Seleccionada
    ----------------------------------------------------------------
     {x1, x2}       {x1} -> {x2}           86%      ?            Si
                    {x2} -> x1               ?      ?             ?
    ----------------------------------------------------------------
     {?, ?}         {?} -> {?}               ?      ?             ?
                    {?} -> {?}               ?      ?             ?
    ----------------------------------------------------------------
     {?, ?}         {?} -> {?}               ?      ?             ?
                    {?} -> {?}               ?      ?             ?
    ----------------------------------------------------------------
     {x1, x2, x3}   {?}    -> {?, ?}         ?      ?             ?
                    {?}    -> {?, ?}         ?      ?             ?
                    {x3}   -> {x1, x2}     67%      ?            No
                    {?, ?} -> {?}            ?      ?             ?
                    {?, ?} -> {?}            ?      ?             ?
                    {?, ?} -> {?}            ?      ?             ?
    ----------------------------------------------------------------
 
---

**Ejercicio.--** Si un cliente compra $x_1$ y $x_3$ que producto o productos se le deben recomendar?

---

---
Sistemas de Recomendación 
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/statistics-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/statistics-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 